In [5]:
from selenium.webdriver import ChromeOptions
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from os import makedirs
from os.path import exists
import bs4 as bs
import pandas as pd
from openpyxl import load_workbook
import time
import os
import json

In [8]:
# Set to developer mode to prevent the use of Selenium from being recognised

options = ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-automation'])
options.add_experimental_option('useAutomationExtension', False)


browser = webdriver.Chrome(executable_path="/Users/isla/Desktop/5507/5507Project/chromedriver3", options=options)
browser.execute_cdp_cmd('Page.addScriptToEvaluateOnNewDocument', {
     'source': 'Object.defineProperty(navigator, "webdriver", {get: () => undefined})'
})


url= 'https://hotel.qunar.com/cn/sanya/?fromDate=2022-11-20&toDate=2022-11-21&cityName=%E4%B8%89%E4%BA%9A'
#Total 50 pages crawled
PageNum = 50 

# Get hotel information

def fetchHotel():
    html=""
    try:
        os.chdir('/Users/isla/Desktop/5507/5507Project')
        browser.get(url)
        time.sleep(3)
        with open('qunar_cookies.txt', 'r', encoding='utf8') as f:
            listCookies = json.loads(f.read())
        for cookie in listCookies:
            cookie_dict = {
                'domain': '.qunar.com',
                'name': cookie.get('name'),
                'value': cookie.get('value'),
                "expires": '',
                'path': '/',
                'httpOnly': False,
                'HostOnly': False,
                'Secure': False
            }
            browser.add_cookie(cookie_dict)  
        browser.refresh()
        html = browser.page_source

    except TimeoutException:
        print("访问超时")
    return html


# Parse html pages and extract data

def parseHtml(html):
    #time.sleep(3)
    html = browser.page_source
    page_html = bs.BeautifulSoup(html, 'html.parser')
    names = page_html.find_all('a',class_="hotel-name")
    types = page_html.find_all(class_="type")
    ratings = page_html.find_all(class_="comm")
    remarks = page_html.find_all(class_="total")
    addresss = page_html.find_all(class_="adress")
    prices = page_html.find_all(class_='price_new')
    TYPE = []
    NAME = []
    RATING = []
    REMARK_NUM = []
    ADDRESS = []
    PRICE = []
    for i in names:
        NAME.append(i.get_text(strip=True))
    for i in types:
        TYPE.append(i.get_text(strip=True))
    for i in ratings:
        RATING.append(i.get_text(strip=True))
    for i in remarks:
        REMARK_NUM.append(i.get_text(strip=True))
    for i in addresss:
        ADDRESS.append(i.get_text(strip=True))
    for i in prices:
        PRICE.append(i.get_text(strip=True))
    return list(zip(NAME,TYPE,RATING,REMARK_NUM,ADDRESS,PRICE))


# Get multiple pages of data

def fetchList():
    html = ""
    try:
        nextPage = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"#root > div > section > section.G_section_1200.clearfix.qunar-assist-screen-product > aside.G_aside_860.fl_left > div.G_page.clearfix > p.next.fl_right.cur.able")))
        nextPage.click()
        time.sleep(6)
    except TimeoutException:
        print("访问超时2")
    return html


# Save file

def saveData(data):
    header = ['NAME', 'TYPE','RATING','REMARK_NUM','ADDRESS','PRICE']
    df = pd.DataFrame(data, columns=header)
    df.to_csv('2sanya_hotel_total.csv', encoding='utf_8_sig', mode='a', index=False, sep=',', header=False )


def main():
    html = fetchHotel()
    for i in range(PageNum):
        jobsData = parseHtml(html)
        saveData(jobsData)
        html = fetchList()  
    browser.quit()


if __name__ == '__main__':
    main()

/var/folders/wt/yyz95v2s7bj68s5bdn7mr34r0000gn/T/ipykernel_16926/3628567128.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path="/Users/isla/Desktop/5507/5507Project/chromedriver3", options=options)
